In [3]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.2602 ,random_state=0 , l1_ratio= 0.501, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 61.88 , C = 0.00010 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_1000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_16_200Samples.csv').iloc[:,-1]
path = "train_latent_50D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_50D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [8]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,26):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 25000)

n_obs , dim =  30, 25
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,75])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1278 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=933038, Mon Apr 13 13:31:42 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -1.796112625680677e+02 1.0e+00 1.18e-01  1e-01  1e-01 0:00.0
    2     26 -1.796117877202401e+02 1.1e+00 1.17e-01  1e-01  1e-01 0:00.0
    3     39 -1.796159325411678e+02 1.1e+00 1.23e-01  1e-01  1e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61126257] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61178772] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1300 -1.967839881600513e+02 1.6e+00 4.54e-03  4e-03  4e-03 0:00.9
  200   2600 -1.968306515128938e+02 1.8e+00 9.13e-05  6e-05  8e-05 0:01.8
  300   3900 -1.968306588551315e+02 1.8e+00 1.20e-06  7e-07  8e-07 0:02.7
  400   5200 -1.968306588567274e+02 1.9e+00 2.03e-08  1e-08  1e-08 0:03.6
  425   5525 -1.968306588567276e+02 2.0e+00 7.81e-09  4e-09  5e-09 0:03.9
termination on tolfun=1e-11 (Mon Apr 13 13:31:47 2020)
final/bestever f-value = -1.968307e+02 -1.968307e+02
incumbent solution: [-0.23908321  0.54882754 -0.08331453 -0.24177621 -0.25010351 -0.19874091
  0.83287729 -0.05434573 ...]
std deviations: [4.25137408e-09 3.91992667e-09 3.94894561e-09 4.74352257e-09
 3.75151857e-09 4.10855886e-09 4.65563054e-09 4.19464528e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=965075, Mon Apr 13 13:31:47 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -1.807846610046242e+02 1.0e+00 1.18e-01  1e-01  1e-01 0:00.0
termination on tol

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.784661] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.18e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.18e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/

  100   1300 -1.937571874561108e+02 1.8e+00 4.44e-03  4e-03  4e-03 0:00.9
  200   2600 -1.937721325165078e+02 1.9e+00 6.41e-05  4e-05  5e-05 0:01.8
  300   3900 -1.937721348809850e+02 1.8e+00 7.42e-07  4e-07  5e-07 0:02.7
  400   5200 -1.937721348814275e+02 1.8e+00 1.06e-08  5e-09  7e-09 0:03.6
  405   5265 -1.937721348814275e+02 1.8e+00 8.93e-09  4e-09  6e-09 0:03.6
termination on tolfun=1e-11 (Mon Apr 13 13:31:52 2020)
final/bestever f-value = -1.937721e+02 -1.937721e+02
incumbent solution: [-0.50486099  0.2452838   0.03023219  0.27211984  0.16192503 -0.07293881
 -0.14273147  0.17804875 ...]
std deviations: [5.30385679e-09 4.51035043e-09 4.39201709e-09 4.90823673e-09
 4.38310858e-09 5.41198894e-09 5.18380925e-09 5.62172579e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=875822, Mon Apr 13 13:31:52 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -1.807846610046242e+02 1.0e+00 1.18e-01  1e-01  1e-01 0:00.0
termination on tol

...

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.784661] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.22e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.22e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/

  100   1300 -1.898099454371362e+02 1.8e+00 9.86e-03  8e-03  1e-02 0:00.9
  200   2600 -1.898910160268552e+02 1.7e+00 1.31e-04  9e-05  1e-04 0:01.8
  300   3900 -1.898910283633283e+02 1.8e+00 1.31e-06  7e-07  9e-07 0:02.7
  400   5200 -1.898910283640181e+02 1.9e+00 1.46e-08  7e-09  9e-09 0:03.6
  414   5382 -1.898910283640181e+02 2.0e+00 8.58e-09  4e-09  5e-09 0:03.8
termination on tolfun=1e-11 (Mon Apr 13 13:33:34 2020)
final/bestever f-value = -1.898910e+02 -1.898910e+02
incumbent solution: [ 0.40461586 -0.00752918  0.1550585  -0.23638785 -0.39452977  0.0635502
  0.03812721 -0.03389589 ...]
std deviations: [4.76438850e-09 4.31249236e-09 4.00931930e-09 4.75966979e-09
 4.35017242e-09 5.03097001e-09 5.02733496e-09 4.84382596e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=873487, Mon Apr 13 13:33:34 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -1.807846610046242e+02 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
termination on tolf

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.784661] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.21e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.21e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/

  100   1300 -1.991960216338377e+02 1.9e+00 7.87e-03  6e-03  8e-03 0:00.9
  200   2600 -1.993023444373264e+02 1.8e+00 8.47e-05  6e-05  7e-05 0:01.8
  300   3900 -1.993023541399264e+02 1.9e+00 1.14e-06  7e-07  8e-07 0:02.7
  400   5200 -1.993023541411505e+02 1.8e+00 3.00e-08  1e-08  2e-08 0:03.6
  428   5564 -1.993023541411509e+02 1.9e+00 8.40e-09  4e-09  5e-09 0:03.9
termination on tolfun=1e-11 (Mon Apr 13 13:33:39 2020)
final/bestever f-value = -1.993024e+02 -1.993024e+02
incumbent solution: [ 0.21196173 -0.23937007  0.06500432  0.08386136 -0.30140855  0.21568155
  0.23566797 -0.14104591 ...]
std deviations: [4.50724704e-09 3.98168343e-09 4.31365822e-09 5.07198551e-09
 4.39411630e-09 4.57661862e-09 4.55637889e-09 4.99510720e-09 ...]
(6_w,13)-aCMA-ES (mu_w=4.0,w_1=38%) in dimension 25 (seed=883947, Mon Apr 13 13:33:39 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 -1.807846610046242e+02 1.0e+00 1.21e-01  1e-01  1e-01 0:00.0
termination on tol

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-180.784661] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.21e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-180.784661, sigma=1.21e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/

In [9]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F16_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:25])
    SVM_Fun [i] = F16(X_Values.iloc[i,25:50])
    ELN_Fun [i] = F16(X_Values.iloc[i,50:75])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-234.34927366]), count=array([1])) 18.047616390820533
SVM
ModeResult(mode=array([-212.57117433]), count=array([1])) 19.501134258268447
ELN
ModeResult(mode=array([-200.31315567]), count=array([1])) 15.59214162854042
